In [5]:
import requests
from bs4 import BeautifulSoup
import re
import rasterio
import pandas as pd
import timeit


In [6]:
# URLs of 
dataset_url = {}
dataset_url["DSM"] = "https://www.geopunt.be/download?container=dhm-vlaanderen-ii-dsm-raster-1m&title=Digitaal%20Hoogtemodel%20Vlaanderen%20II,%20DSM,%20raster,%201m"
dataset_url["DTM"] = "https://www.geopunt.be/download?container=dhm-vlaanderen-ii-dtm-raster-1m&title=Digitaal%20Hoogtemodel%20Vlaanderen%20II,%20DTM,%20raster,%201m"

dataset_url

{'DSM': 'https://www.geopunt.be/download?container=dhm-vlaanderen-ii-dsm-raster-1m&title=Digitaal%20Hoogtemodel%20Vlaanderen%20II,%20DSM,%20raster,%201m',
 'DTM': 'https://www.geopunt.be/download?container=dhm-vlaanderen-ii-dtm-raster-1m&title=Digitaal%20Hoogtemodel%20Vlaanderen%20II,%20DTM,%20raster,%201m'}

In [7]:
for data_type in dataset_url.keys():
    print(data_type)

DSM
DTM


In [10]:
for data_type in dataset_url.keys():
    r = requests.get(dataset_url[data_type])
    soup = BeautifulSoup(r.content, "html")
    download_elem = soup.find_all("div", attrs={"class": "downloadfileblock"})[0]
    link_elems = download_elem.find_all("a")
    for link_elem in link_elems:
        print(link_elem.get("href"))

https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k01.zip
https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k02.zip
https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k03.zip
https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k04.zip
https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k05.zip
https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k06.zip
https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k07.zip
https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k08.zip
https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k09.zip
https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k10.zip
https://downloadagiv

In [73]:
zip_links = []
for elem in soup.find_all("li"):
    zip_links.append(elem.find_all("a")[0].get("href"))
    
zip_links


# for elem in soup.select("li.search-results__item a.card__title-link"):
        #href_listing = elem.get("href")
        #index_search = href_listing.find("?searchId")

['https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k01.zip',
 'https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k02.zip',
 'https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k03.zip',
 'https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k04.zip',
 'https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k05.zip',
 'https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k06.zip',
 'https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k07.zip',
 'https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k08.zip',
 'https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k09.zip',
 'https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVI

In [55]:
start_time = timeit.default_timer()  # start the timer
stop_times = []

bounds = []
profiles = []
for link in zip_links:
    print(link)
    
    m = re.search("/dhm-vlaanderen-ii-dsm-raster-1m/(?P<zip_file_name>[\w\-]+).zip$", link)
    zip_file_name = m.group("zip_file_name")
    print(zip_file_name)
    tif_request = "zip+" + link + "!/GeoTIFF/" + zip_file_name + ".tif"
    print(tif_request)
    
    # img_DTM = rasterio.open(tif_request)
    with rasterio.open(tif_request) as img_DTM:
        print(img_DTM.bounds)
    
    bounds.append(img_DTM.bounds)
    profiles.append(img_DTM.profile)
    
    stop_time = timeit.default_timer()
    stop_times.append(stop_time)
    #/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k01.zip
#img_DTM = rasterio.open('zip+https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k01.zip!/GeoTIFF/DHMVIIDSMRAS1m_k01.tif')
#img_DTM.bounds

stop_time = timeit.default_timer()
print(f"Total time: {stop_time-start_time:.2f}")


https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k01.zip
DHMVIIDSMRAS1m_k01
zip+https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k01.zip!/GeoTIFF/DHMVIIDSMRAS1m_k01.tif
BoundingBox(left=145000.0, bottom=238000.0, right=162000.0, top=247000.0)
https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k02.zip
DHMVIIDSMRAS1m_k02
zip+https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k02.zip!/GeoTIFF/DHMVIIDSMRAS1m_k02.tif
BoundingBox(left=162000.0, bottom=238000.0, right=194000.0, top=250000.0)
Total time: 170.36


In [58]:
for i in range(0,5):
    img_DTM = rasterio.open(tif_request)
    print(i)

0
1
2
3
4


In [59]:
img_DTM

<open DatasetReader name='zip+https://downloadagiv.blob.core.windows.net/dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k02.zip!/GeoTIFF/DHMVIIDSMRAS1m_k02.tif' mode='r'>

In [60]:
# Can we "partially" open?
help(rasterio.open)

Help on function open in module rasterio:

open(fp, mode='r', driver=None, width=None, height=None, count=None, crs=None, transform=None, dtype=None, nodata=None, sharing=False, **kwargs)
    Open a dataset for reading or writing.
    
    The dataset may be located in a local file, in a resource located by
    a URL, or contained within a stream of bytes.
    
    In read ('r') or read/write ('r+') mode, no keyword arguments are
    required: these attributes are supplied by the opened dataset.
    
    In write ('w' or 'w+') mode, the driver, width, height, count, and dtype
    keywords are strictly required.
    
    Parameters
    ----------
    fp : str, file object or pathlib.Path object
        A filename or URL, a file object opened in binary ('rb') mode,
        or a Path object.
    mode : str, optional
        'r' (read, the default), 'r+' (read/write), 'w' (write), or
        'w+' (write/read).
    driver : str, optional
        A short format driver name (e.g. "GTiff" or "

In [8]:
bounds = src.open(tif_request)
bounds.left
bounds.right
bounds.bottom
bounds.top

[BoundingBox(left=145000.0, bottom=238000.0, right=162000.0, top=247000.0),
 BoundingBox(left=162000.0, bottom=238000.0, right=194000.0, top=250000.0)]

In [9]:
f'{1:05d}'

'00001'

In [10]:
# Check if DTM files have the same boundaries
# Load them in a dataframe

In [117]:
img_DTM.bounds.left

162000.0

In [66]:
img_DTM.profile

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': -9999.0, 'width': 32000, 'height': 12000, 'count': 1, 'crs': CRS.from_epsg(31370), 'transform': Affine(1.0, 0.0, 162000.0,
       0.0, -1.0, 250000.0), 'blockxsize': 512, 'blockysize': 512, 'tiled': True, 'compress': 'deflate', 'interleave': 'band'}

In [12]:
df = pd.DataFrame({"URL": zip_links})
df["Boundaries"] = bounds

ValueError: Length of values (2) does not match length of index (43)

In [ ]:
# Save metadata in a dataframe

In [ ]:
# Save the dataframe to a csv file
df.to_csv("./assets/meta_data_blocks.csv", index=False)

In [36]:
import sys
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

                 BeautifulSoup:  2.0 KiB
                           _ii: 1004.0 B
                          _i29: 1004.0 B
                          _i34: 1004.0 B
                           _i5:  950.0 B
                          _i21:  672.0 B
                           obj:  667.0 B
                          _i20:  667.0 B
                          _i22:  667.0 B
                          _i23:  667.0 B
